The first example of using the Agent Based Modeling (ABM) mechanism of SuperNeuro, called SuperNeuroABM. Here, we will create a simple network of three neurons, an input, a hidden and an output neuron, connected in a feedforward manner. We will apply input spikes to neuron 0, and observe spikes from all the neurons in the network.

In [1]:
#!pip uninstall -y superneuroabm
#!pip install git+https://github.com/ornl/superneuroabm@dev
#!pip install matplotlib

In [2]:
from superneuroabm.model import NeuromorphicModel

In [3]:
# Configure the simulation parameters:
sim_time = 100
neuron_threshold = 1.0
neuron_reset = -1.0
neuron_refractory = 1.0


snn_model = NeuromorphicModel(use_cuda=True)

neurons = {"input": None, "hidden": None, "output": None}

for neuron_name in neurons.keys():
    neurons[neuron_name] = snn_model.create_neuron(
        threshold=neuron_threshold,
        reset_state=neuron_reset,
        refractory_period=neuron_refractory,
    )

print(neurons)

{'input': 0, 'hidden': 1, 'output': 2}


In [4]:
# Create Synpase:
snn_model.create_synapse(neurons["input"], neurons["hidden"], 
            synaptic_delay = 5,
            weight = 4,
        )

snn_model.create_synapse(neurons["hidden"], neurons["output"],    # TODO: Chathika can you check why the neuron_h spikes are not available at end of simulation
            synaptic_delay = 3,                  
            weight = 2,
        )

In [6]:
# Apply Spikes:
snn_model.spike(neurons["input"], 0, 2)    # (node_id, spike_time, spike_value)
snn_model.spike(neurons["input"], 10, 4)

# Add the input spikes to the common list:
network_spikes = []
network_spikes.append([0, 10])

# Setup the model
snn_model.setup()

#Begin simulation:
snn_model.simulate(ticks=sim_time, update_data_ticks=1)

/home/co1/miniconda3/envs/superneuro_release/lib/python3.9/site-packages/numba/cuda/compiler.py:726: NumbaPerformanceWarning: Grid size (1) < 2 * SM count (60) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [7]:
# Monitor the output spikes from all neurons:
in_spikes = snn_model.get_spikes(neurons["input"])
h_spikes = snn_model.get_spikes(neurons["hidden"])
out_spikes = snn_model.get_spikes(neurons["output"])

print(neurons["input"], neurons["hidden"], neurons["output"])

0 1 2


In [9]:
# Plot the spike raster:
#from superneuroabm.core.agent import *

import matplotlib.pyplot as plt

for neuron_id in neurons.values():
    network_spikes.append(snn_model.get_spikes(neuron_id))

colors1 = ['b', 'k', 'k', 'k']  #Input spikes in blue and network spikes in black
fig, axs = plt.subplots(1,1,figsize=(10,5))
axs.eventplot(network_spikes,colors=colors1,lineoffsets=1,linelengths=0.1,linewidths=2.0)
plt.title('Spike Raster')
plt.xlabel('Timesteps')
plt.ylabel('Neuron index')
plt.tight_layout()
#plt.gca().invert_yaxis()

TypeError: 'dict_values' object cannot be interpreted as an integer